In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import pandas as pd
import numpy as np           
import numpy
from numpy import array
# import matplotlib.pyplot as plt
import math
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [3]:
#code for MAPE, referred from the url: https://scikit-learn.org/stable/modules/model_evaluation.html#mean-absolute-percentage-error
#'eps' is an arbitrary small yet strictly positive number to avoid undefined results when y is zero.
def mean_absolute_percentage_error(y_true, y_pred):                                        
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    eps=0.01
    for i in range(len(y_true)):
      if y_true[i]==0.00:
        y_true[i]=eps
    return np.mean((np.abs(y_true - y_pred)) / np.abs(y_true)) * 100

In [4]:
x=pd.read_csv('np5k.csv',parse_dates=['Date'],index_col='Date')
x = x.loc["2016-01-01" :]                         #Since 2016-01-01, 5y(1234rows till 2020-12-31), + year 2021's rows (till 30th of April)
y=x.copy()                                        #deep copy
x.reset_index(drop=True, inplace=True)
x

,3MINDIA,AIAENG,APLAPOLLO,AARTIDRUGS,AARTIIND,ABBOTINDIA,ADANIENT,ADANITRANS,AEGISCHEM,AKZOINDIA,...,VMART,VIPIND,VSTIND,VAIBHAVGBL,VENKEYS,VINATIORGA,VOLTAS,WHIRLPOOL,WIPRO,ZYDUSWELL
0,11321.950195,867.168823,144.383698,144.525604,248.652145,5673.990234,47.666069,41.049999,102.688416,1264.629517,...,614.104858,96.934898,1446.978149,459.678467,324.586823,224.686325,302.245453,638.013733,202.975861,830.219421
1,10935.799805,847.834595,145.164093,142.959702,246.615967,5707.625000,45.020885,38.650002,102.446335,1249.578613,...,581.427551,93.276970,1441.762329,449.980774,320.425507,222.553543,290.931915,640.025696,203.432037,825.170898
2,10784.349609,838.550293,152.120255,143.263153,247.941849,5656.812012,46.872513,38.799999,106.029037,1238.562622,...,576.304688,93.652122,1441.804810,451.901978,323.020233,220.133163,288.423126,648.319153,203.140244,831.517761
3,10644.599609,825.293945,150.328232,142.607681,244.485062,5565.869629,45.629280,38.349998,101.429634,1227.818604,...,562.875549,93.980408,1437.309814,449.843506,321.404633,217.736801,289.417175,661.716553,202.483627,830.652161
4,10673.750000,802.753357,144.846161,136.611053,241.927979,5443.218262,42.296364,37.150002,100.316071,1219.159790,...,539.001709,89.994217,1447.190186,444.400055,307.696564,213.015900,272.801941,652.932251,200.568375,817.910583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309,26181.150391,1900.400024,1287.650024,715.599976,1492.050049,15052.849609,1173.349976,1055.199951,286.000000,2214.350098,...,2616.800049,335.000000,3318.649902,821.739990,1525.900024,1666.349976,946.150024,2174.949951,480.299988,2162.699951
1310,25505.300781,1905.650024,1313.099976,723.150024,1562.900024,15202.599609,1191.150024,1044.099976,290.799988,2229.100098,...,2630.399902,333.000000,3309.199951,830.440002,1526.300049,1725.750000,976.400024,2161.050049,485.049988,2102.800049
1311,25674.300781,1920.150024,1321.150024,719.299988,1539.199951,15172.900391,1185.199951,1056.550049,313.350006,2240.600098,...,2645.699951,347.549988,3289.500000,860.880005,1552.050049,1720.199951,962.150024,2145.750000,489.299988,2123.899902
1312,25494.349609,1880.150024,1330.150024,718.750000,1556.949951,14997.250000,1182.300049,1050.150024,308.549988,2224.949951,...,2704.199951,357.649994,3201.300049,851.000000,1572.949951,1689.349976,967.099976,2171.500000,489.850006,2096.199951


In [5]:
stonks=[]
for i in x:
  stonks.append(i)
len(stonks)

184

In [6]:
alldata=x   #the original dataset

In [7]:
timesteps=84                                     #lstm hyperparameters "Subject to be tuned"
epoch=100
batchSize=32
ineurons=186
hneurons=186
after2020=len(y.loc["2021-01-01" : ])                    #number of days after 31-12-2020 "automated"

In [8]:
def create_dataset(dataset, time_step=1):         # convert an array of values into a dataset matrix which will be used to train the lstm model.
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step):
		a = dataset[i:(i+time_step), 0]               #i=0, 0,1,2,3-----(timesteps-1)  -> timesteps
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [9]:
def forcast(df1, timesteps, epoch, batchSize, ineurons, hneurons, after2020):             
  scaler=MinMaxScaler(feature_range=(0,1))   
  df1=scaler.fit_transform(np.array(df1).reshape(-1,1))           #minmax scalar transformation of data

  before_2021_data_length=int(len(df1)-after2020)                 #length of data before 2021
  training_size=int(before_2021_data_length*0.80)                 #80% of training size, refered from Yadav et al (2020) (Science Direct)
  train_data=df1[0:training_size,:]                              
  test_data=df1[training_size:before_2021_data_length,:1]         #20% of testing data, refered from Yadav et al (2020) (Science Direct)                
  inpdata=df1[before_2021_data_length-timesteps:len(df1),:1]      #getting the data from 01-01-2021 onwards


  #reshape into X=t,t+1,t+2,t+3,........t+"timestep-1" and Y=t+"timestep"
  X_train, y_train = create_dataset(train_data, timesteps)
  x_inp, y_inp = create_dataset(inpdata, timesteps)
  x_test, y_test = create_dataset(test_data,timesteps)

  X_train = X_train.reshape(X_train.shape[0],X_train.shape[1] , 1) 
  x_inp = x_inp.reshape(x_inp.shape[0],x_inp.shape[1] , 1)        #reshape input to be [samples, time steps, features] which is required for LSTM
  x_test = x_test.reshape(x_test.shape[0],x_test.shape[1] , 1)

  # initialising stacked lstm
  model=Sequential()
  model.add(LSTM(ineurons,return_sequences=True,input_shape=(timesteps,1),activation='tanh', dropout=0.1))
  model.add(LSTM(hneurons,return_sequences=True,activation='tanh', dropout=0.1))
  model.add(LSTM(hneurons,return_sequences=True,activation='tanh', dropout=0.1))
  model.add(LSTM(hneurons,return_sequences=True,activation='tanh', dropout=0.1))
  model.add(LSTM(hneurons,return_sequences=True,activation='tanh', dropout=0.1))
  model.add(LSTM(hneurons,activation='tanh', dropout=0.1))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss='mean_squared_error',optimizer='adam')

  model.fit(X_train,y_train,validation_data=(x_test,y_test),epochs=epoch,batch_size=batchSize,verbose=1)     # training of the model
  
  test_predict=model.predict(x_test)                    #prediction using test data as input

  #performance metrics between, original test data and predicted test data
  msetst =mean_squared_error(y_test,test_predict)
  rmsetst=math.sqrt(msetst)
  maetst =mean_absolute_error(y_test,test_predict)
  r2tst  =r2_score(y_test,test_predict)
  mapetst=mean_absolute_percentage_error(y_test,test_predict)
  tstlst =[msetst,rmsetst,maetst,r2tst,mapetst]


  #model is trained again on the test data so as to increase the learning (it is often termed as incremental learning)
  #refered from url: https://www.justintodata.com/forecast-time-series-lstm-with-tensorflow-keras/#step-2-transforming-the-dataset-for-tensorflow-keras
  #refered from url: https://github.com/keras-team/keras/issues/4446
  model.fit(x_test,y_test,epochs=epoch,batch_size=batchSize,verbose=1)
  
  out_predict=model.predict(x_inp)                      #dynamic prediction of the stock's closing price from 01-01-2021 onwards

  #performance metrics between, original data(after 31-12-2020) and dynamically predicted data (after 31-12-2020)
  mseinp =mean_squared_error(y_inp,out_predict)
  rmseinp=math.sqrt(mseinp)
  maeinp =mean_absolute_error(y_inp,out_predict)
  r2inp  =r2_score(y_inp,out_predict)
  mapeinp=mean_absolute_percentage_error(y_inp,out_predict)
  inplst =[mseinp,rmseinp,maeinp,r2inp,mapeinp]


  lst=[]
  for i in out_predict:
    lst.append(i)

  p=train_data.tolist()
  q=test_data.tolist()
  p.extend(q)                                         #appending train and test data to make dataset before 2021 (data till 31-12-2020)
  p.extend(lst)                                       #appending the data, forcasted from 01-01-2021 onwards, to the data till 31-12-2020
  p=scaler.inverse_transform(p).tolist()

  return pd.DataFrame(p), tstlst, inplst
  #returns a dataframe, tstlst => test performance metrics, inplst => forcasted data performance metrics
    

In [ ]:
mtest=[]
mdynamic=[]
fdata=pd.DataFrame()
prog=1
for i in alldata:                                   # this for loop will be iterated for 42 times i.e. for each column of the original dataset
  temp=alldata[i]
  print(prog)
  prog=prog+1
  ftemp,trmse,drmse=forcast(temp, timesteps, epoch, batchSize, ineurons, hneurons, after2020)    #hyperparameters are provided as input here
  fdata = pd.concat([fdata,ftemp],axis = 1)
  mtest.append(trmse)
  mdynamic.append(drmse)
fdata.columns=stonks

1
Epoch 1/100
29/29 [==============================] - 28s 646ms/step - loss: 0.0173 - val_loss: 0.0059
Epoch 2/100
29/29 [==============================] - 17s 601ms/step - loss: 0.0044 - val_loss: 0.0082
Epoch 3/100
29/29 [==============================] - 17s 605ms/step - loss: 0.0041 - val_loss: 0.0066
Epoch 4/100
29/29 [==============================] - 17s 589ms/step - loss: 0.0038 - val_loss: 0.0053
Epoch 5/100
29/29 [==============================] - 17s 601ms/step - loss: 0.0035 - val_loss: 0.0048
Epoch 6/100
29/29 [==============================] - 17s 594ms/step - loss: 0.0033 - val_loss: 0.0057
Epoch 7/100
29/29 [==============================] - 17s 602ms/step - loss: 0.0032 - val_loss: 0.0086
Epoch 8/100
29/29 [==============================] - 17s 598ms/step - loss: 0.0034 - val_loss: 0.0065
Epoch 9/100
29/29 [==============================] - 17s 598ms/step - loss: 0.0037 - val_loss: 0.0055
Epoch 10/100
29/29 [==============================] - 18s 612ms/step - loss: 0.0

Epoch 80/100
29/29 [==============================] - 18s 622ms/step - loss: 6.9730e-04 - val_loss: 0.0014
Epoch 81/100
29/29 [==============================] - 18s 612ms/step - loss: 6.8160e-04 - val_loss: 9.5548e-04
Epoch 82/100
29/29 [==============================] - 18s 620ms/step - loss: 5.8971e-04 - val_loss: 7.5534e-04
Epoch 83/100
29/29 [==============================] - 18s 615ms/step - loss: 6.1205e-04 - val_loss: 8.0331e-04
Epoch 84/100
29/29 [==============================] - 18s 609ms/step - loss: 6.2126e-04 - val_loss: 8.0115e-04
Epoch 85/100
29/29 [==============================] - 18s 615ms/step - loss: 5.8532e-04 - val_loss: 0.0013
Epoch 86/100
29/29 [==============================] - 18s 614ms/step - loss: 6.2818e-04 - val_loss: 0.0014
Epoch 87/100
29/29 [==============================] - 18s 610ms/step - loss: 5.7089e-04 - val_loss: 0.0017
Epoch 88/100
29/29 [==============================] - 19s 653ms/step - loss: 6.7853e-04 - val_loss: 0.0017
Epoch 89/100
29/29 [=

6/6 [==============================] - 3s 484ms/step - loss: 5.1704e-04
Epoch 71/100
6/6 [==============================] - 3s 524ms/step - loss: 4.5651e-04
Epoch 72/100
6/6 [==============================] - 3s 467ms/step - loss: 4.6240e-04
Epoch 73/100
6/6 [==============================] - 3s 499ms/step - loss: 4.2468e-04
Epoch 74/100
6/6 [==============================] - 3s 470ms/step - loss: 4.3039e-04
Epoch 75/100
6/6 [==============================] - 3s 509ms/step - loss: 4.2534e-04
Epoch 76/100
6/6 [==============================] - 3s 468ms/step - loss: 3.6580e-04
Epoch 77/100
6/6 [==============================] - 3s 489ms/step - loss: 4.3960e-04
Epoch 78/100
6/6 [==============================] - 3s 506ms/step - loss: 3.2532e-04
Epoch 79/100
6/6 [==============================] - 3s 489ms/step - loss: 2.5801e-04
Epoch 80/100
6/6 [==============================] - 3s 479ms/step - loss: 5.4409e-04
Epoch 81/100
6/6 [==============================] - 3s 474ms/step - loss: 5.19

29/29 [==============================] - 18s 619ms/step - loss: 0.0010 - val_loss: 0.0093
Epoch 56/100
29/29 [==============================] - 18s 613ms/step - loss: 9.6201e-04 - val_loss: 0.0080
Epoch 57/100
29/29 [==============================] - 18s 614ms/step - loss: 9.0629e-04 - val_loss: 0.0073
Epoch 58/100
29/29 [==============================] - 18s 615ms/step - loss: 9.0656e-04 - val_loss: 0.0068
Epoch 59/100
29/29 [==============================] - 18s 615ms/step - loss: 9.0857e-04 - val_loss: 0.0086
Epoch 60/100
29/29 [==============================] - 18s 615ms/step - loss: 9.4226e-04 - val_loss: 0.0093
Epoch 61/100
29/29 [==============================] - 18s 612ms/step - loss: 8.4078e-04 - val_loss: 0.0063
Epoch 62/100
29/29 [==============================] - 18s 610ms/step - loss: 9.1622e-04 - val_loss: 0.0053
Epoch 63/100
29/29 [==============================] - 18s 615ms/step - loss: 9.4729e-04 - val_loss: 0.0090
Epoch 64/100
29/29 [==============================] - 

6/6 [==============================] - 3s 473ms/step - loss: 8.9012e-04
Epoch 41/100
6/6 [==============================] - 3s 484ms/step - loss: 7.5721e-04
Epoch 42/100
6/6 [==============================] - 3s 471ms/step - loss: 7.2960e-04
Epoch 43/100
6/6 [==============================] - 3s 487ms/step - loss: 8.7860e-04
Epoch 44/100
6/6 [==============================] - 3s 483ms/step - loss: 7.6408e-04
Epoch 45/100
6/6 [==============================] - 3s 464ms/step - loss: 7.5484e-04
Epoch 46/100
6/6 [==============================] - 3s 474ms/step - loss: 7.1898e-04
Epoch 47/100
6/6 [==============================] - 3s 498ms/step - loss: 6.7681e-04
Epoch 48/100
6/6 [==============================] - 3s 486ms/step - loss: 7.2317e-04
Epoch 49/100
6/6 [==============================] - 3s 473ms/step - loss: 6.6421e-04
Epoch 50/100
6/6 [==============================] - 3s 487ms/step - loss: 6.2616e-04
Epoch 51/100
6/6 [==============================] - 3s 487ms/step - loss: 6.62

...

6/6 [==============================] - 4s 667ms/step - loss: 5.6860e-04
Epoch 6/100
6/6 [==============================] - 4s 674ms/step - loss: 5.3184e-04
Epoch 7/100
6/6 [==============================] - 4s 664ms/step - loss: 6.2514e-04
Epoch 8/100
6/6 [==============================] - 4s 671ms/step - loss: 0.0010
Epoch 9/100
6/6 [==============================] - 4s 665ms/step - loss: 8.9745e-04
Epoch 10/100
6/6 [==============================] - 4s 668ms/step - loss: 6.6602e-04
Epoch 11/100
6/6 [==============================] - 4s 674ms/step - loss: 6.0367e-04
Epoch 12/100
6/6 [==============================] - 4s 665ms/step - loss: 7.1082e-04
Epoch 13/100
6/6 [==============================] - 4s 668ms/step - loss: 7.3910e-04
Epoch 14/100
6/6 [==============================] - 4s 673ms/step - loss: 6.6145e-04
Epoch 15/100
6/6 [==============================] - 4s 671ms/step - loss: 6.6958e-04
Epoch 16/100
6/6 [==============================] - 4s 678ms/step - loss: 7.1787e-04
E

29/29 [==============================] - 37s 926ms/step - loss: 0.0234 - val_loss: 0.0205
Epoch 2/100
29/29 [==============================] - 24s 828ms/step - loss: 0.0094 - val_loss: 0.0145
Epoch 3/100
29/29 [==============================] - 24s 839ms/step - loss: 0.0075 - val_loss: 0.0084
Epoch 4/100
29/29 [==============================] - 24s 838ms/step - loss: 0.0078 - val_loss: 0.0081
Epoch 5/100
29/29 [==============================] - 24s 840ms/step - loss: 0.0070 - val_loss: 0.0086
Epoch 6/100
29/29 [==============================] - 24s 839ms/step - loss: 0.0061 - val_loss: 0.0064
Epoch 7/100
29/29 [==============================] - 24s 841ms/step - loss: 0.0059 - val_loss: 0.0065
Epoch 8/100
29/29 [==============================] - 24s 841ms/step - loss: 0.0058 - val_loss: 0.0054
Epoch 9/100
29/29 [==============================] - 24s 838ms/step - loss: 0.0052 - val_loss: 0.0053
Epoch 10/100
29/29 [==============================] - 24s 840ms/step - loss: 0.0048 - val_loss

29/29 [==============================] - 28s 980ms/step - loss: 0.0011 - val_loss: 0.0033
Epoch 81/100
29/29 [==============================] - 30s 1s/step - loss: 0.0012 - val_loss: 0.0047
Epoch 82/100
29/29 [==============================] - 29s 1s/step - loss: 9.5999e-04 - val_loss: 0.0069
Epoch 83/100
29/29 [==============================] - 27s 919ms/step - loss: 8.0650e-04 - val_loss: 0.0044
Epoch 84/100
29/29 [==============================] - 26s 901ms/step - loss: 8.7908e-04 - val_loss: 0.0044
Epoch 85/100
29/29 [==============================] - 28s 980ms/step - loss: 8.1239e-04 - val_loss: 0.0035
Epoch 86/100
29/29 [==============================] - 27s 945ms/step - loss: 8.0228e-04 - val_loss: 0.0038
Epoch 87/100
29/29 [==============================] - 26s 908ms/step - loss: 8.3510e-04 - val_loss: 0.0047
Epoch 88/100
29/29 [==============================] - 29s 1s/step - loss: 7.4872e-04 - val_loss: 0.0027
Epoch 89/100
29/29 [==============================] - 26s 906ms/ste

6/6 [==============================] - 6s 926ms/step - loss: 6.2639e-04
Epoch 73/100
6/6 [==============================] - 6s 917ms/step - loss: 6.3219e-04
Epoch 74/100
6/6 [==============================] - 6s 928ms/step - loss: 5.8638e-04
Epoch 75/100
6/6 [==============================] - 6s 906ms/step - loss: 6.0864e-04
Epoch 76/100
6/6 [==============================] - 6s 909ms/step - loss: 6.2052e-04
Epoch 77/100
6/6 [==============================] - 6s 929ms/step - loss: 5.2364e-04
Epoch 78/100
6/6 [==============================] - 6s 926ms/step - loss: 5.9623e-04
Epoch 79/100
6/6 [==============================] - 6s 911ms/step - loss: 5.6389e-04
Epoch 80/100
6/6 [==============================] - 6s 911ms/step - loss: 7.2299e-04
Epoch 81/100
6/6 [==============================] - 6s 919ms/step - loss: 7.1669e-04
Epoch 82/100
6/6 [==============================] - 6s 927ms/step - loss: 7.3620e-04
Epoch 83/100
6/6 [==============================] - 6s 922ms/step - loss: 7.39

29/29 [==============================] - 33s 1s/step - loss: 0.0049 - val_loss: 0.0304
Epoch 59/100
29/29 [==============================] - 32s 1s/step - loss: 0.0046 - val_loss: 0.0304
Epoch 60/100
29/29 [==============================] - 33s 1s/step - loss: 0.0049 - val_loss: 0.0254
Epoch 61/100
29/29 [==============================] - 33s 1s/step - loss: 0.0048 - val_loss: 0.0330
Epoch 62/100
29/29 [==============================] - 33s 1s/step - loss: 0.0050 - val_loss: 0.0299
Epoch 63/100
29/29 [==============================] - 33s 1s/step - loss: 0.0050 - val_loss: 0.0243
Epoch 64/100
29/29 [==============================] - 33s 1s/step - loss: 0.0048 - val_loss: 0.0306
Epoch 65/100
29/29 [==============================] - 33s 1s/step - loss: 0.0047 - val_loss: 0.0302
Epoch 66/100
29/29 [==============================] - 33s 1s/step - loss: 0.0060 - val_loss: 0.0301
Epoch 67/100
 1/29 [>.............................] - ETA: 32s - loss: 0.0055

In [ ]:
fdata # dataset with 2021 rows forcasted dynamically

In [ ]:
alldata # dataset with original 2021 rows

In [ ]:
fdata.to_csv('data_inc/fdata.csv')   #dataset saved in .csv format

In [ ]:
clm=['MSE','RMSE','MAE','R2','MAPE']
pd.DataFrame(mtest,index=stonks,columns=clm).to_csv('data_inc/mtest.csv') #metric values saved 

In [ ]:
pd.DataFrame(mdynamic,index=stonks,columns=clm).to_csv('data_inc/mdynamic.csv') #metric values saved 